In [2]:
import re
import sys
import time
from multiprocessing import Process, Value, Array, Lock
import execute
import execution
import pre_operation
import stack_operator

codes=[]
com=[]
opr=[]
stack=[]
rstack=[]
temp=[]
ldata=[]
rdata=[]
top=-1
count_pc=0
parflag=0
command_file='inv_code.txt'
command_file2='inv_code.txt'
command_mode='b'
command_mode2='v'


        
if __name__ == '__main__':
    start_time = time.time()
    start=[]
    end=[]
    tabledata=[]
    tablecount= Value('i',0)
    address = Array('i',10)
    value = Array('i',10)
    rstack = Array('i',100000)
    lstack = Array('i',100000)
    rtop = Value('i',0)
    ltop = Value('i',0)
    endflag={}
    endflag0=Value('i',0)
    lock={}
    
    mlock = Lock()
    lockfree  = Lock()
    a='1'
    path='table.txt'
    f=open(path,mode='r')
    tabledata=f.read()
    f.close()
    k=0
    #変数名と変数の値との関係を保存するための機能現在の処理には必要ない将来用
    #for i in range(0,len(tabledata),20):
    #    t=tabledata[i+11:i+13]
    #    s=re.search(r'\d+',t)
    #    address[k]=((int)(s.group()))
    #    t2=tabledata[i+13:i+19]
    #    s2=re.search(r'\d+',t2)
    #    value[k]=((int)(s2.group()))
    #    k=k+1
    #    tablecount=tablecount+1

    (ldata,rdata,stack)=pre_operation.backward(ldata,rdata,stack,command_mode)

    if command_mode=='code':
        print("convert into inv_code.txt")
        (start,end,codes,com,opr,count_pc,parflag)=pre_operation.coderead(start,end,codes,com,opr,count_pc,parflag,command_file)
        pre_operation.forward(ltop.value,rtop.value,com,opr,lstack,rstack,value,count_pc,command_mode,command_file,command_file2)
        sys.exit()
        
    #forward mode
    if command_mode=='f':
        (start,end,codes,com,opr,count_pc,parflag)=pre_operation.coderead(start,end,codes,com,opr,count_pc,parflag,command_file)
        for i in range(0,parflag,1):
            endflag[i] = Value('i',0)
            lock[i] = Lock()
        if parflag!=0:
            if command_mode2=='q':
                mode='1'
            else:
                mode=input('mode   1:auto 2:select >> ')
            #逐次実行の部分を実行
            stack=execution.execution(mode,lockfree,lockfree,com,opr,0,start[0],stack,address,value,tablecount,rstack,lstack,rtop,ltop,endflag0,0,command_mode,command_mode2)
            if mode=='2':
                for i in range(0,parflag,1):
                    lock[i].acquire()
                process={}
                #並列プロセスの生成
                for i in range(0,parflag,1):        
                    process[i]=Process(target=execution.execution,args=(mode,lock[i],mlock,com,opr,start[i],end[i],stack,address,value,tablecount,rstack,lstack,rtop,ltop,endflag[i],i+1,command_mode,command_mode2))
            if mode=='1':
                process={}
                #並列プロセスの生成
                for i in range(0,parflag,1):        
                    process[i]=Process(target=execution.execution,args=(mode,lock[0],mlock,com,opr,start[i],end[i],stack,address,value,tablecount,rstack,lstack,rtop,ltop,endflag[i],i+1,command_mode,command_mode2))
            for i in range(0,parflag,1):
                    process[i].start()
            #モニター: メインプロセスでサブプロセスの実行を管理する
            if mode=='2':
                while a!='esc':
                    a=input('>> ')
                    mlock.acquire(False)
                    ifflag=0
                    for i in range(0,parflag,1):
                        if a==str(i+1) and endflag[i].value!=1 and ifflag==0:
                            ifflag=1
                            lock[i].release()
            for i in range(0,parflag,1):
                process[i].join()
            for i in range(0,tablecount.value,1):
                stack[i]=value[i]
            execution.execution(mode,lockfree,lockfree,com,opr,end[parflag-1]+1,len(com),stack,address,value,tablecount,rstack,lstack,rtop,ltop,endflag0,0,command_mode,command_mode2)
        elif parflag==0:
            mode='1'
            execution.execution(mode,lockfree,mlock,com,opr,0,len(com),stack,address,value,tablecount,rstack,lstack,rtop,ltop,endflag0,0,command_mode,command_mode2)
        pre_operation.forward(ltop.value,rtop.value,com,opr,lstack,rstack,value,count_pc,command_mode,command_file,command_file2)

    elif command_mode=='b':
        (start,end,codes,com,opr,count_pc,parflag)=pre_operation.coderead(start,end,codes,com,opr,count_pc,parflag,command_file)
        for i in range(0,parflag,1):
            endflag[i] = Value('i',0)
            lock[i] = Lock()
        for i in range(0,len(ldata),1):
            lstack[i]=int(ldata[i])
            ltop.value=ltop.value+1
        for i in range(0,len(rdata),1):
            rstack[i]=int(rdata[i])
            rtop.value=rtop.value+1
        #for i in range(0,len(stack),1):
            #value[i]=int(stack[i])
        ltop.value=ltop.value-1
        rtop.value=rtop.value-1
        mode='0'
        if parflag!=0:
            #______measure time mode_________ 
            if command_mode2=='q':
                mode='1'
            else:
                mode=input('mode   1:auto  2:select >> ')
            for i in range(0,parflag,1):
                lock[i].acquire()
            process={}
            execution.execution(mode,lockfree,lockfree,com,opr,0,end[0],stack,address,value,tablecount,rstack,lstack,rtop,ltop,endflag0,0,command_mode,command_mode2)
            #並列プロセスの生成
            for i in range(0,parflag,1):
                process[i]=Process(target=execution.execution,args=(mode,lock[parflag-i-1],mlock,com,opr,end[i],start[i],stack,address,value,tablecount,rstack,lstack,rtop,ltop,endflag[i],parflag-i,command_mode,command_mode2))
            for i in range(0,parflag,1):
                process[i].start()
            a='2'
            if mode=='2':
                while a!='esc':
                    a=input('process '+str(lstack[ltop.value])+' ')
                    mlock.acquire(False)
                    for i in range(0,parflag,1):
                        if int(lstack[ltop.value])==i+1:
                            lock[i].release()
            elif mode=='1':
                while a!='esc':
                    mlock.acquire()
                    ifflag=0
                    endcount=0
                    for i in range(0,parflag,1):
                        if int(lstack[ltop.value])==i+1 and ifflag==0:
                            lock[i].release()
                            ifflag=1
                    for i in range(0,parflag,1):
                        if endflag[i].value==1:
                            endcount=endcount+1
                    if endcount==parflag:
                        a='esc'
            for i in range(0,parflag,1):
                process[i].join()
            #逐次実行の部分を実行
            execution.execution(mode,lockfree,lockfree,com,opr,start[parflag-1]+1,count_pc,stack,address,value,tablecount,rstack,lstack,rtop,ltop,endflag0,0,command_mode,command_mode2)
        elif parflag==0:
            execution.execution(mode,lockfree,lockfree,com,opr,0,len(com),stack,address,value,tablecount,rstack,lstack,rtop,ltop,endflag0,0,command_mode,command_mode2)
    elif command_mode=='c':
        (start,end,codes,com,opr,count_pc,parflag)=pre_operation.coderead(start,end,codes,com,opr,count_pc,parflag,command_file)
        pre_operation.forward(ltop.value,rtop.value,com,opr,lstack,rstack,value,count_pc,command_mode,command_file,command_file2)
    #経過時間の表示
    elapsed_time = time.time()-start_time
    print("elapsed_time:{0}".format(elapsed_time) + "[sec]")

mode   1:auto  2:select >> 1
~~~~~~~~Process0 execute~~~~~~~~

pc = 1   command =   alloc   operand = 0

shared variable stack: [0]

~~~~~~~~Process0 execute~~~~~~~~

pc = 2   command =   alloc   operand = 1

shared variable stack: [0, 0]

~~~~~~~~Process0 execute~~~~~~~~

pc = 3   command =   alloc   operand = 2

shared variable stack: [0, 0, 0]

~~~~~~~~Process0 execute~~~~~~~~

pc = 56   command = restore   operand = 2

shared variable stack: [0, 1, 0]

~~~~~~~~Process0 execute~~~~~~~~

pc = 57   command =     nop   operand = 0

shared variable stack: [0, 1, 0]

~~~~~~~~Process0 execute~~~~~~~~

pc = 58   command = restore   operand = 1

shared variable stack: [0, 0, 0]

~~~~~~~~Process0 execute~~~~~~~~

pc = 59   command =     nop   operand = 0

shared variable stack: [0, 0, 0]

~~~~~~~~Process0 execute~~~~~~~~

pc = 60   command = restore   operand = 0

shared variable stack: [0, 0, 0]

~~~~~~~~Process0 execute~~~~~~~~

pc = 61   command =     nop   operand = 0

shared variable st

In [2]:
result=open("result.txt",'r')
r=result.read()
print(r)
result.close()

In [1]:
result=open("result.txt",'w')
result.write('')
result.close()